In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import sqlite3
import datetime


In [ ]:
# import nltk
# nltk.download('stopwords')

In [ ]:
# nltk.download('punkt')

In [ ]:
category_name = 'Спортивные костюмы мужские'

In [ ]:
# min_price = 5000

In [ ]:
DF = pd.read_sql(sql,connect)

## bad comments selection

In [ ]:
bad_items = DF[(DF["stars"]<=3) | ((DF["disadvantage"].apply(len)>=10) & (DF["stars"]<4)) |  
               ((DF["advantage"].apply(len)<=1) & (DF["comment"].apply(len)<=1) & DF["disadvantage"].apply(len)>=2)]

bad_items.loc[bad_items['disadvantage'].apply(len)<=5,'disadvantage']=bad_items['comment']

bad_items=bad_items[bad_items['disadvantage'].apply(len)>=3]

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

In [ ]:
# ! pip install -U spacy>=3.0

In [ ]:
# ! python -m spacy download ru_core_news_md

In [ ]:
import ru_core_news_md
nlp = ru_core_news_md.load()

In [ ]:
from nltk.tokenize import word_tokenize
import re
import string

table = str.maketrans('', '', string.punctuation)

def cleaner(text):
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    words = [w for w in words if not w in stop_words]
    
    return words

bad_items['disadvantage_raw']=bad_items['disadvantage'].copy()
bad_items['disadvantage'] = bad_items['disadvantage'].apply(cleaner)
bad_items['disadvantage'] = bad_items['disadvantage'].apply(lambda x: ' '.join(x))

In [ ]:
# ! pip install numpy==1.17.4

In [ ]:
# ! pip install pandas==0.25.3

In [ ]:
import swifter

In [ ]:
def nlp_format(x):
    return  [w.lemma_.lower() for w in nlp(x)]

In [ ]:
bad_items['disadvantage_stem'] = bad_items['disadvantage'].swifter.apply(nlp_format,axis=1)

Pandas Apply:   0%|          | 0/2391 [00:00<?, ?it/s]

In [ ]:
bad_items['disadvantage_stem'] = bad_items['disadvantage_stem'].apply(\
    lambda x: [word for word in x if re.search('[а-яА-Я]', word)])

# bad_items['disadvantage_stem'] = bad_items['disadvantage_stem'].apply(lambda x: [word for word in x if not re.search('[0-9]', word)])
bad_items['disadvantage_stem'] = bad_items['disadvantage_stem'].apply(lambda x: ' '.join(x))

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


In [ ]:
bad_items=bad_items.sample(frac=1)

In [ ]:
from tqdm import tqdm
import random

rows_raw={}
rows_corpus={}
total=bad_items.shape[0]

all_rows = list(range(bad_items.shape[0]))

vals_development =[1]*round(total*0.1) + [0]
critical_len=round(total*0.05)

i1_len=0
for i1 in all_rows:
    
    i1_len+=1
    
    i1_stem=bad_items.iloc[i1]['disadvantage_stem']
    rows_raw[i1]=[]
    rows_raw[i1].append((2,bad_items.iloc[i1]['disadvantage_raw']))
    rows_corpus[i1]=i1_stem.split()
    
    print(f"row: {i1}, left_count: {len(all_rows)}, pct_loss: {round(vals_development[-1],4)}")
    
    initial_row_count = len(all_rows)
    
    for j in all_rows:
        j_stem=bad_items.iloc[j]['disadvantage_stem']
        
        c=cosine_sim(i1_stem,j_stem)
        
        if c>0.25:
            all_rows.remove(j)
            rows_raw[i1].append((round(c,3),bad_items.iloc[j]['disadvantage_raw']))
            rows_corpus[i1]+=j_stem.split()
    
    rows_corpus[i1] = list(set(rows_corpus[i1]))
    final_row_count = len(all_rows)
    vals_development+=[(initial_row_count/final_row_count)-1]
    
            
    if len(all_rows)<total*0.3 or max(vals_development[-critical_len:])<0.05 or i1_len/final_row_count>=0.6:
        break


row: 0, left_count: 2391, pct_loss: 0
row: 2, left_count: 2380, pct_loss: 0.0046
row: 4, left_count: 2272, pct_loss: 0.0475
row: 6, left_count: 2232, pct_loss: 0.0179
row: 8, left_count: 2226, pct_loss: 0.0027
row: 10, left_count: 2216, pct_loss: 0.0045
row: 12, left_count: 2194, pct_loss: 0.01
row: 14, left_count: 2188, pct_loss: 0.0027
row: 16, left_count: 2177, pct_loss: 0.0051
row: 20, left_count: 2167, pct_loss: 0.0046
row: 22, left_count: 2156, pct_loss: 0.0051
row: 24, left_count: 2153, pct_loss: 0.0014
row: 26, left_count: 2124, pct_loss: 0.0137
row: 28, left_count: 2117, pct_loss: 0.0033
row: 32, left_count: 2114, pct_loss: 0.0014
row: 34, left_count: 2098, pct_loss: 0.0076
row: 36, left_count: 2065, pct_loss: 0.016
row: 39, left_count: 2059, pct_loss: 0.0029
row: 41, left_count: 1940, pct_loss: 0.0613
row: 43, left_count: 1938, pct_loss: 0.001
row: 45, left_count: 1932, pct_loss: 0.0031
row: 48, left_count: 1924, pct_loss: 0.0042
row: 51, left_count: 1920, pct_loss: 0.0021
ro

In [ ]:
# raw text
rows_sorted = dict(sorted(rows_raw.items(), key=lambda x: x[0]) )
rows_groups = {key_: list(set([x[1] for x in rows_sorted[key_]])) for key_ in rows_sorted.keys()}
rows_groups = sorted(rows_groups.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
com_groups_raw=pd.DataFrame(rows_groups).reset_index(drop=True).rename(columns={0:"id",1:"raw_text"})[["id","raw_text"]]
com_groups_raw

,id,raw_text
0,39,"[Ни размер, ни цвет не совпадают с действитель..."
1,2,"[качество так себе, Качество так себе, Ткань о..."
2,129,"[голимая синтетика, да нще и маломерит на разм..."
3,76,"[Не очень качественно сшит, начес очень линяет..."
4,91,"[Где штаны?!, штаны длиннее заявленных, Маломе..."
...,...,...
145,28,[Внутри костюма подкладка вся скатывается.Это ...
146,22,"[Брала мужу, подойдёт на высокого мужчину, на ..."
147,12,[Ширпотреб! Тёплый - зто да! В остальном каках...
148,6,[прислали не ту модель( с надписью и на капюшо...


In [ ]:
# corpus
rows_sorted = dict(sorted(rows_corpus.items(), key=lambda x: x[0]) )
rows_groups = sorted(rows_sorted.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
com_groups_corpus=pd.DataFrame(rows_groups).reset_index(drop=True).rename(columns={0:"id",1:"corpus"})[["id","corpus"]]
com_groups_corpus

,id,corpus
0,39,"[заказывать, маленький, качественный, верх, по..."
1,76,"[спереди, широкий, куртка, троечку, использова..."
2,129,"[куртка, широкий, впритык, изображение, верх, ..."
3,249,"[комплект, жилет, вещь, тянуть, заявить, отвра..."
4,295,"[таблица, широкий, вернуть, нормально, сюда, о..."
...,...,...
145,171,"[брак, прийти]"
146,141,"[липнуть, нем]"
147,359,[одноразовый]
148,99,[тонковот]


In [ ]:
result = com_groups_raw.merge(com_groups_corpus,how='left',on="id")
result = result[result["raw_text"].apply(len)>5]

In [ ]:
result["raw_text"]=result["raw_text"].apply(lambda x: [str(w).lower() for w in x])
result["raw_text"] =result["raw_text"].apply(set)
result["raw_text"] =result["raw_text"].apply(list)s

In [ ]:
result.loc[3]["corpus"]

['спереди',
 'широкий',
 'куртка',
 'троечку',
 'использовать',
 'вместо',
 'само',
 'красится',
 'фото',
 'соответствую',
 'человек',
 'одежда',
 'очень',
 'нитка',
 'заказывала',
 'длинный',
 'полотно',
 'хаки',
 'растягиваться',
 'сшить',
 'цвет',
 'блёклый',
 'тонкий',
 'хороший',
 'соответствовать',
 'сзади',
 'тёмный',
 'плотный',
 'тяжёлый',
 'штаны',
 'прийти',
 'толстовка',
 'зад',
 'капюшон',
 'подклад',
 'натяжка',
 'весь',
 'костюм',
 'лезть',
 'швв',
 'материал',
 'ворсинка',
 'большой',
 'отличаться',
 'заказать',
 'размера',
 'разный',
 'сильно',
 'тот',
 'тёплый',
 'начёс',
 'ровный',
 'линять',
 'изумруд',
 'качество',
 'качественно',
 'мятый',
 'рюкзак',
 'торчать',
 'размер',
 'ткань',
 'стирка',
 'карман',
 'штанах']

In [ ]:
result.explode('raw_text')

,id,raw_text,corpus
0,39,кофта очень сильно маломерит! не понимаю зачем...,"[заказывать, маленький, качественный, верх, по..."
0,39,не соответствует размеру. на 50 размер взяла 5...,"[заказывать, маленький, качественный, верх, по..."
0,39,размер маловат,"[заказывать, маленький, качественный, верх, по..."
0,39,материал и размер,"[заказывать, маленький, качественный, верх, по..."
0,39,несоответствие размеру,"[заказывать, маленький, качественный, верх, по..."
...,...,...,...
12,8,тонкий,"[дорого, тонкий, зацепка, мяться, материал, ле..."
12,8,"дорого, очень тонкий материал, мнётся!","[дорого, тонкий, зацепка, мяться, материал, ле..."
12,8,очень тонкий материал.,"[дорого, тонкий, зацепка, мяться, материал, ле..."
12,8,"материал тонкий, легко поставить зацепки","[дорого, тонкий, зацепка, мяться, материал, ле..."


In [ ]:
result.to_csv("comment_group.csv")

In [ ]:
result.explode('raw_text').to_csv("comment_group_expl.csv")